In [2]:
import numpy
import numpy as np
import os
import pandas
import cv2
from tqdm import tqdm
import random

import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, Lambda, AveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [3]:
IMG_SIZE = 256
sigmaX = 30

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[numpy.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][numpy.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][numpy.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = numpy.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [4]:
aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
dr_df = pandas.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")

In [5]:
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
dr_dir = "../input/diabetic-retinopathy-resized/resized_train/resized_train/"

images = []
labels = []
count0 = 0

# for idx in tqdm(range(len(dr_df))):
#     if dr_df.iloc[idx]['level'] == 0:
#         if count0 < 6038:
#             images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#             labels.append(dr_df.iloc[idx]['level'])
#             count0 += 1
#     else:
#         images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#         labels.append(dr_df.iloc[idx]['level'])

for idx in tqdm(range(len(aptos_df))):
    images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png'))
    labels.append(aptos_df.iloc[idx]['diagnosis'])

100%|██████████| 3662/3662 [13:28<00:00,  4.53it/s]


In [6]:
images = numpy.array(images)
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

images = images.astype('float32') / 255.0
labels = labels.astype('float32')
images = images.reshape(images.shape[0], IMG_SIZE, IMG_SIZE,3)

In [7]:
input_shape = (IMG_SIZE, IMG_SIZE, 3)
num_classes = 5

In [8]:
def euclid_distance(vects):
    x,y = vects
    sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))
 
def euclid_distance_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [9]:
def contrastive_loss(y_true, y_pred):
    y_true=tf.dtypes.cast(y_true, tf.float64)
    y_pred=tf.dtypes.cast(y_pred, tf.float64)
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [10]:
def create_pairs(x, digit_indices):
    pairs = []
    labels = []

    n=min([len(digit_indices[d]) for d in range(num_classes)]) -1

    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1,0]
    return np.array(pairs), np.array(labels)

In [11]:
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)
 
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [12]:
def create_base_net(input_shape):
    input_ = Input(shape = input_shape)
    x = Conv2D(4, (5,5), activation = 'relu')(input_)
    x = AveragePooling2D(pool_size = (2,2))(x)
    x = Conv2D(4, (5,5), activation = 'relu')(input_)
    x = MaxPooling2D(pool_size = (2,2))(x)
    
    x = Conv2D(4, (5,5), activation = 'relu')(input_)
    x = AveragePooling2D(pool_size = (2,2))(x)
    x = Conv2D(16, (5,5), activation = 'relu')(x)
    x = AveragePooling2D(pool_size = (2,2))(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation = 'tanh')(x)
    model = Model(input_, x)
    #   model.summary()

    return model

In [13]:
digit_indices = [np.where(labels == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(images, digit_indices) 
base_network = create_base_net(input_shape)

NameError: name 'inputs_' is not defined

In [ ]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
 
processed_a = base_network(input_a)
processed_b = base_network(input_b)
 
distance = Lambda(euclid_distance, output_shape=euclid_distance_output_shape)([processed_a, processed_b])
 
model = Model([input_a, input_b], distance)
#train
model.compile(loss=contrastive_loss, optimizer='adam', metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y, batch_size=128, epochs= 16, validation_split=0.1)